# CS 182: Artificial Intelligence
# Assignment 2: Adversarial Search and Constraint Satisfaction
* Fall 2017
* Due: Oct 5, 5pm

In Part 1 (15 points), you will implement adversarial search algorithms to help Pacman find paths in a maze with ghosts. Note: We will use the Pacman framework developed at Berkeley. This framework is used world-wide to teach AI, therefore it is very important that you DO NOT publish your solutions online. 

In Part 2 (15 points), you will implement a Sudoku solver by treating Sudoku as a constraint satisfaction problem (CSP). 

In Part 3 (10 points), you will answer 3 written questions. 

## Submission Requirements

Solutions should be **submitted to Canvas**. For the written part, format your responses as a PDF named `lastname_firstname_HW2.pdf` and submit it to Canvas. For the computational part, submit the link to your private Github repo and the name of the person you worked with as a comment on the PDF submission (if you work in a pair). **The written part of this assignment must be done individually**. Submissions not following the requirements *will not be graded*.

## Part 1: Adversarial Search (15 points)

Follow the instructions **for Q2-Q4** at: 

> http://ai.berkeley.edu/multiagent.html

The page includes questions requiring implementation of some of the game playing algorithms we studied in class. The updated Berkeley files are in the multiagent folder.

Do not use Assignment 1’s files because there are some small changes. You can, however, use your search.py and searchAgents.py files if they are helpful.

## Part 2: Solving Sudoku as a Constraint Satisfaction Problem (CSP) (15 Points)

In a traditional search problem (as in HW1), we treat each state of a problem abstractly. A state has a goal test and a transition model, but we never look inside the representation of a state. In a constraint satisfaction problem (CSP), a state is no longer opaque, we can peek in to check if we are on the right track. 

In this problem set, you will implement a very common CSP, Sudoku! Sudoku solvers leverage a few essential techniques for solving CSPs. The code for this project is in `sudoku.py`.

This runnable IPython notebook walks you through the elements of Sudoku you will need to implement for this assignment.

In [1]:
from sudoku import *
import IPython.display
sudoku = Sudoku(boardHard)

(If you want to run the assignment in IPython notebook, you should call `set_args([])` to set global  `set_args(["-forward", "1"])`)

## Modeling (5 points)

### Problem 0: Introduction to Sudoku
If you are not familiar with Sudoku puzzles, read the Wikipedia page on Sudoku to familiarize yourself with the basic rules.

https://en.wikipedia.org/wiki/Sudoku

Here is an example of a Sudoku grid:

In [3]:
IPython.display.HTML(sudoku.printhtml())

,,,,,8,9,,2
6,,4,3,,,,,
,,,5,9,,,,
,,5,7,,3,,,9
7,,,,4,,,,
,,9,,,,3,,5
,8,,,,4,,,
,4,1,,,,,3,
2,,,1,5,,,,


We also implement a method `prettyprint` which will show the true state of the CSP. Each column shows the labels that still can be used in each column and row (note we do not show the remaining values in the boxes). You will first need to implement the factor updates below to get prettyprint to work.

Note here we show the web version of this function `prettyprinthtml` (you can also use `print sudoku` from the command line.)

In [4]:
IPython.display.HTML(sudoku.prettyprinthtml())

1,1,,,1,1,1,1,1,,,,,,,,,
,2,2,2,2,2,2,2,,,,,,,,,,
3,3,3,,3,,,,3,,,,,,,,,
4,,,4,,,4,4,4,,,,,,,,,
5,5,,,,5,5,5,,,,,,,,,,
,6,6,6,6,6,6,6,6,,,,,,,,,
,7,7,,7,7,7,7,7,,,,,,,,,
8,,8,8,8,,8,8,8,,,,,,,,,
9,9,,9,,9,,9,,,,,,,,,,
,,,,,8,9,,2,1,,3,4,5,6,7,,
6,,4,3,,,,,,1,2,,,5,,7,8,9


To begin, you will want to be familiar with the Sudoku board and interface for accessing rows, columns, and boxes. 

In [8]:
sudoku.board

[[0, 0, 0, 0, 0, 8, 9, 0, 2],
 [6, 0, 4, 3, 0, 0, 0, 0, 0],
 [0, 0, 0, 5, 9, 0, 0, 0, 0],
 [0, 0, 5, 7, 0, 3, 0, 0, 9],
 [7, 0, 0, 0, 4, 0, 0, 0, 0],
 [0, 0, 9, 0, 0, 0, 3, 0, 5],
 [0, 8, 0, 0, 0, 4, 0, 0, 0],
 [0, 4, 1, 0, 0, 0, 0, 3, 0],
 [2, 0, 0, 1, 5, 0, 0, 0, 0]]

In [6]:
print sudoku.row(1)
print sudoku.col(3)
print sudoku.box(0)

[6, 0, 4, 3, 0, 0, 0, 0, 0]
[0, 3, 5, 7, 0, 0, 0, 0, 1]
[0, 0, 0, 6, 0, 4, 0, 0, 0]


### Problem 1: CSP Variables

For this assignment we will be modelling Sudoku as a CSP. Each box in the grid represents a variable based on `(row, col)`. The variable is either assigned a label ($1\ldots 9$) or $\epsilon$ (`0`) when it has not yet been assigned. Given the current assignment, the *domain* of each variable is also limited. When all the variables have been assigned the assignment is complete. For the first problem you should implement the following functions in the `Sudoku` class which model the variables of the Sudoku CSP. 

In [5]:
doc(sudoku.firstEpsilonVariable)

In [6]:
doc(sudoku.complete) 

In [7]:
doc(sudoku.variableDomain) 

### Problem 2: CSP Factors

Next we will implement the factors of the CSP. The rules of Sudoku say that there must be labels from 1-9 in each row, column, and box. Each of these will be represented by factors `(type, id)`, for instance `(ROW, 2)` is the factor corresponding to the third-row. 

_Note:_ These functions should update `self.factorRemaining` and `self.factorNumConflicts`. The datatype of `self.factorRemaining[type, id]` should always be a list of length 9. When a label is no longer available, instead of removing it, it should be replaced by `None`. For conflicts, you should count the number of times any label is used more than once. We provide the helper function `crossOff` to help with this book-keeping.

For this problem, you should implement functions which keep track of the remaining labels available for a given factor as well as the number of violation of that factor in the case of inconsistent assignments. To do this, you should implement the following functions:

In [3]:
doc(sudoku.updateFactor)

In [4]:
doc(sudoku.updateAllFactors)

In [2]:
doc(sudoku.updateVariableFactors)

## Classical Search (5 points)

### Problem 3: Solving Sudoku with backtracking

The `solveCSP` function will simply perform a depth first search on a tree of generic problem states. Running this function requires getting the next variable to search and all the possible labels that variable can take on. 

First you should implement the function `getSuccessors` which should return a list of Sudoku objects representing all possible successor assignments resulting from assigning a label to a variable. Note that for simplicity, unlike the pseudo-code in class we are not doing backtracking (undoing the assignments). This function will need to call `setVariable` which copies the state to produce new assignments.

In [8]:
doc(sudoku.getSuccessors)

After you have implemented this method, run the program by running.

In [10]:
#!python sudoku.py 
# or to see each stage use
#!python sudoku.py --debug=1

Take note of how many states the CSP solver explores, and how much time it takes to solve the puzzle.

### Problem 4: Improving performance with Forward Checking.

Now, you will try to improve the performance of your Sudoku solver by implementing forward checking. Forward checking cuts off search when any variable has an empty domain. There are two ways to implement forward checking.

1. Whenever a variable is assigned an update, the domain of any associated variable (i.e. any variable two edges away in the factor graph) is updated as well. 

2. Recompute the domains of all unassigned variables on-the-fly to try and find empty variables. 

The first method is faster, but requires some more modifications and variable tracking. The second method is simpler, and fine for the Sudoku problem. We will accept both.

First, take a look at: the function `getSuccessorsWithForwardChecking()` and then implement `forwardCheck`


In [9]:
doc(sudoku.forwardCheck)

Run the program and take note of the number of states explored and how much time it takes.


In [12]:
#!python sudoku.py --forward 1

## Local Search (5 points)

### Problem 5: Sampling Complete Solutions

In the next several problems we consider a different approach to finding a solution to Sudoku, local search. In 
local search instead of working with consistent, incomplete assignments, we will instead use inconsistent, complete assignments. To start we need to sample a random complete assignment. It is often good to start with some randomness but also satisfying some of the factors. For Sudoku we start with the following constraint:

* Sample returns a complete assignment with all *Row* factors satisfied.


You should implement this with the function `randomRestart`:

In [11]:
doc(sudoku.randomRestart)

When this works you should be able to call `prettyprint` and see zeros along the rows but not the columns, for example

In [16]:
board = [[4, 2, 5, 1, 7, 8, 9, 3, 6], [7, 4, 6, 3, 5, 2, 1, 9, 8], [1, 2, 8, 5, 7, 4, 9, 3, 6], [2, 1, 8, 6, 9, 3, 5, 4, 7], [3, 1, 2, 6, 9, 8, 5, 7, 4 
], [1, 5, 6, 7, 4, 9, 2, 3, 8], [9, 4, 5, 3, 8, 1, 6, 7, 2], [3, 8, 2, 9, 7, 1, 5, 6, 4], [9, 1, 8, 4, 3, 6, 7, 5, 2]]
IPython.display.HTML(Sudoku(board).prettyprinthtml())

,,1,,1,,,1,1,,,,,,,,,
,,,2,2,,,2,,,,,,,,,,
,3,3,,,,3,,3,,,,,,,,,
,,4,,,,4,,,,,,,,,,,
5,,,,,5,,,5,,,,,,,,,
6,6,,,6,,,,,,,,,,,,,
,7,7,,,7,,,,,,,,,,,,
8,,,8,,,8,8,,,,,,,,,,
,9,9,,,,,,9,,,,,,,,,
4,2,5,1,7,8,9,3,6,,,,,,,,,
7,4,6,3,5,2,1,9,8,,,,,,,,,


### Problem 6: Neighbors

Local search algorithms also require being able to produce neighbors for a given assignment. We will be doing this by stochastic descent, so we will never be required to fully enumerate all the neighbors. Instead we will find neighboring assignments at random.

One way to produce neighbors is to change variables at random. However for Sudoku this hardly ever results in progress towards a consistent solution. For Sudoku we can be a bit more clever and maintain consistency along some of the factors, in particular the row factors. 

To produce partially consistent neighbors, we do the following swap:

* Randomly select a row 
* Swap two of the entries, being careful not to change any of the original values. 

For this problem you will implement this function as `randomSwap`

In [12]:
doc(sudoku.randomSwap)

### Problem 7: Stochastic Descent

Finally, you will implement the full local search algorithm. The algorithm should start with a random assignment with  consistent rows. At each iteration we run the following check:

1. Sample a neighbor

2. If it has a better or equal score under $f$, then move to that neighbor

3. Otherwise, return to step 1.



For the scoring function $f$ we use the current number of constraints that are violated. For efficiency, our implementation keeps a running count of the violations of each assignment. You will have to understand this representation and implement the following function.

In [13]:
doc(sudoku.gradientDescent)

To run the algorithm, we use the following commandline. Note that for this problem we will use the easier sudoku board as local search is less effective than standard DFS.

In [19]:
#!python sudoku.py --easy 1 --localsearch=1

Run this algorithm 5 times. How often does it find a solution? Instrument your code so that it prints out the number of constraints still being violated. 

Now modify the stochastic descent function to randomly take some non-optimal moves. 

1. Sample a neighbor

2. If it has a better or equal score under $f$, then move to that neighbor

3. With probability 0.001, then move to that neighbor even if $f$ is higher.

4. Otherwise, return to step 1.

Run this algorithm 5 times on the problem. What do you see now? 

## Part 3: Written Assignment (10 points)

### Question 1 (3 points)

In addition to minimax/expectimax, another way to design an agent to play Pacman is to create an agent that uses an evaluation function to decide its actions. This means that at each step, the agent uses an evaluation function to rate each possible action (i.e. a direction that Pacman can move) and chooses the action with the best rating. Describe an evaluation function that could be used to play Pacman successfully.

### Question 2 (3 points)

Prove the following assertion: For every game tree, the utility obtained by MAX using minimax decisions against a suboptimal MIN will be never be lower than the utility obtained playing against an optimal MIN. Can you come up with a game tree in which MAX can do still better using a _suboptimal_ strategy against a suboptimal MIN?

### Question 3 (4 points)

AC-3 puts back on the queue _every_ arc $(X_k, X_i)$ whenever _any_ value is deleted from the domain of $X_i$, even if each value of $X_k$ is consistent with several remaining values of $X_i$. Suppose that, for every arc $(X_k, X_i)$, we keep track of the number of remaining values of $X_i$ that are consistent with each value of $X_k$. Explain how to update these numbers efficiently and hence show that arc consistency can be enforced in total time $O(n^2d^2)$.